In [ ]:
import numpy as np
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import plotly.graph_objs as go
import plotly_arrows as plar
init_notebook_mode(connected=True)

In [ ]:
c = 3e8 # Speed of light

## Simple boundary example

First, let's just have normal incidence, with lines to show the propagation direction.

<img title="Nomally incident EM wave" src="normal-incidence.png" width="602px">

The equations governing this interaction are

$$\mathbf{E_i} = E_{i,0} e^{i (\mathbf{k}_i \cdot \mathbf{r} - \omega_i t)}, \qquad \mathbf{E_t} = E_{t,0} e^{i (\mathbf{k}_t \cdot \mathbf{r} - \omega_t t)}, \qquad \mathbf{E_r} = E_{r,0} e^{i (\mathbf{k}_i \cdot \mathbf{r} - \omega_r t)}$$

N.B. the reflected wave propagates in the $-z$ direction

Taking the dielectric boundary to be at $z=0$, and applying continuity of $E_\parallel$ across the boundary,

$$E_i + E_r = E_t$$
$$\Rightarrow E_{i,0} e^{-i \omega_i t} + E_{r,0} e^{-i \omega_r t} + E_{t,0} e^{-i \omega_i t}$$

For the waves to be phase-matched at all times, the time-varying parts must always be equal, which can only happen if

$$E_{i,0} + E_{r,0} = E_{t,0}$$

Note that the wavenumber $k$ is not generally the same across the boundary.

As $B = \frac{k}{\omega} E$, the B field continuity is

$$B_{i,0} - B_{r,0} = B_{t,0}$$

But, $B = \frac{\eta}{c}E$

$$\Rightarrow \eta_1 E_{i,0} - \eta_1 E_{r,0} = \eta_2 E_{t,0}$$

So the ratio of transmitted to incident field is

$$t = \frac{E_{t,0}}{E_{i,0}} = \frac{\eta_1}{\eta_1 + \eta_2}$$

And the ratio of reflected to incident field is

$$r = \frac{E_{r,0}}{E_{i,0}} = \frac{\eta_1 - \eta_2}{\eta_1 + \eta_2}$$

Reflected ray satisfies $$\theta_i = \theta_r$$

Transmitted ray satisfies Snell's law $$n_1 \sin{\theta_i} = n_2 \sin{\theta_t}$$

In [ ]:
class Wave:
    def __init__(self, theta, phi, out, E_0, w, polarisation, n1, width=5, color="rgb(0,0,0)"):
        """
        Args:
            theta (float) - [radians] [0, π]
            phi (float) - [radians] [0, 2π]
            out (bool) - True if outgoing, False if incoming (to the origin)
            
            E (float) - Magnitude of the E field
            w (float) - Angular frequency
            polarisation - TODO
            n1 (float) - The incident material's refractive index
            
            width (int) - line thickness
            color (str) - [hex/rgb] line color
        """
        self.theta = theta
        self.phi = phi
        self.out = out
        self.arrow = plar.Arrow(theta=self.theta, phi=self.phi, out=self.out, color=color)
        
        self.E_0 = E_0
        self.w = w
        self.k = (n1 * w) / c
        self.B_0 = (self.k / self.w) * E_0
        self.polarisation = polarisation
        self.n1 = n1
        
        self.width = width
        self.color = color
    
    def transmit(self, n2):
        """
        Args:
            n2 (float) - The dielectric material's refactive index
        """
        self.n2 = n2
        
        E_t0 = self.E_0 * (self.n1 / (self.n1 + self.n2))
        w = self.w
          
        theta_t = np.pi + np.arcsin((self.n1 / self.n2) * np.sin(self.theta)) 
        
        return Wave(theta=theta_t, phi=self.phi, out=True, E_0=E_t0, w=w, polarisation=None, n1=self.n2, color="#0099FF")
        
    def reflect(self, n2):
        
        E_r0 = self.E_0 * ((self.n1 - self.n2) / (self.n1 + self.n2))
        w = self.w # Bit of a fudge
        
        theta_r = -self.theta
        
        return Wave(theta=theta_r, phi=self.phi, out=True, E_0=E_r0, w=w, polarisation=None, n1=self.n1)


In [ ]:
Incident = Wave(theta=np.deg2rad(41.8), phi=0, out=False, E_0=2., w=(2*np.pi*550e12), polarisation=None, n1=1.)
print("Incident:", Incident.E_0, Incident.k, Incident.w, Incident.B_0, Incident.n1)
Transmitted = Incident.transmit(n2=1.5)
print("Transmitted:", Transmitted.E_0, Transmitted.k, Transmitted.w, Transmitted.B_0, Transmitted.n1)
Reflected = Incident.reflect(n2=1.4)
print("Reflected:", Reflected.E_0, Reflected.k, Reflected.w, Reflected.B_0, Reflected.n1)

In [ ]:
plot_data = Incident.arrow.data + Transmitted.arrow.data + Reflected.arrow.data

layout = {
    'autosize': True,
    'scene': {
        'aspectmode': 'cube',
        'xaxis': {'range': [-1, 1], 'autorange': False, 'zeroline': True},
        'yaxis': {'range': [-1, 1], 'autorange': False, 'zeroline': True},
        'zaxis': {'range': [-1, 1], 'autorange': False, 'zeroline': True},
        'camera': {
            'up': {'x': 0, 'y': 1, 'z': 0} # DOESN'T WORK -- WHY NOT!?
        }
    }
}

fig = go.Figure(data=plot_data, layout=layout)
iplot(fig)